In [5]:
from distutils.log import error
from genericpath import exists
from turtle import end_fill
from open_crypto import runner
from matplotlib import pyplot as plt
import sqlite3 
import pandas as pd
import datetime


import os

if 'RDS_HOSTNAME' in os.environ:
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.mysql',
            'NAME': os.environ['RDS_DB_NAME'],
            'USER': os.environ['RDS_USERNAME'],
            'PASSWORD': os.environ['RDS_PASSWORD'],
            'HOST': os.environ['RDS_HOSTNAME'],
            'PORT': os.environ['RDS_PORT'],
        }
    }

#create database
depth_db = pd.DataFrame(columns=['exchange', 'mid price', 'spread', 'depth 50 bps','depth 50 USD', 'depth 100 bps','depth 100 USD', 'depth 150 bps','depth 150 USD','depth 200 bps','depth 200 USD', 'time'])

In [10]:
def calculate_depth(database, exchange, range): #depths are calculated in BPS
    range = range / 100 #convert to kBPS (1000BPS = 1KBPS)
    #get midprice of the last range of trades
    price = prices.loc[prices['exchange'] == exchange, 'price'].item()

    #calculate depth boundaries
    depth_asks = price * (1 + range)
    depth_bids = price - (price * range)

    #get trades within the depth boundaries
    trades = database.loc[(database['bids_price'] > depth_bids) & (database['asks_price'] < depth_asks)]
    trades = trades.assign(**{'price': trades['bids_price'].fillna(trades['asks_price'])})
    

    #calculate depth
    depth = trades['price'].std()

    return depth

def calculate_spread(database, exchange):
    #get midprice of the last range of trades
    bidmaxprice = database.loc[database['exchange'] == exchange, 'bids_price'].max()
    askminprice = database.loc[database['exchange'] == exchange, 'asks_price'].min()

    #calculate spread
    spread = bidmaxprice - askminprice

    return spread

def fetch_data():
    for exchange in orderbook['exchange'].unique():
        depth_val_50 = calculate_depth(orderbook, exchange, 50)
        depth_val_100 = calculate_depth(orderbook, exchange, 100)
        depth_val_150 = calculate_depth(orderbook, exchange, 150)
        depth_val_200 = calculate_depth(orderbook, exchange, 200)
        spread = calculate_spread(orderbook, exchange)
        depth_val_50_USD = depth_val_50 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        depth_val_100_USD = depth_val_100 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        depth_val_150_USD = depth_val_150 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        depth_val_200_USD = depth_val_200 * prices.loc[prices['exchange'] == exchange, 'price'].item()
        row = [exchange, prices.loc[prices['exchange'] == exchange, 'price'].item(), spread, depth_val_50, depth_val_50_USD, depth_val_100, depth_val_100_USD, depth_val_150, depth_val_150_USD, depth_val_200, depth_val_200_USD, cur_time] 
        depth_db.loc[len(depth_db)] = row

def check_dbstatus():
    if con.total_changes == 0:
        return(print("DB connection established"))
    else:
        return(print("failed to establish connection to database"))



In [11]:
#fetch data from exchanges
#fetch data from exchanges
runner.run('amkt.yaml')
# Read sqlite query results into a pandas DataFrame

con = sqlite3.connect("amkt_data.db")

check_dbstatus()

orderbook = pd.read_sql_query('SELECT * FROM order_books_view;', con)
prices = pd.read_sql_query('SELECT exchange, MAX(price) AS price FROM trades_view GROUP BY exchange;', con)
# Verify that result of SQL query is stored in the dataframe
#get the time of the snapshot
cur_time = datetime.datetime.now() 
fetch_data()

con.close()

Requesting 2 exchange(s) for job: orderbook.                                    
Requesting 2 exchange(s) for job: tickers.
Requesting data... 50.00 % \                                                    Pair-ID 253 - Coinbase: 18340 tuple(s)
Pair-ID 661 - Ftx: 100 tuple(s)
Terminating.
Pair-ID 253 - Coinbase: 1000 tuple(s)                                           
Pair-ID 661 - Ftx: 100 tuple(s)
Terminating.
DB connection established


In [13]:
depth_db

,exchange,mid price,spread,depth 50 bps,depth 50 USD,depth 100 bps,depth 100 USD,depth 150 bps,depth 150 USD,depth 200 bps,depth 200 USD,time
0,COINBASE,1523.49,109.32,148.991143,226986.516401,236.561154,360398.551835,391.979656,597177.085489,430.729848,656212.616249,2022-07-26 09:33:16.743594
1,FTX,1523.30,109.10,148.938662,226878.263322,236.476902,360225.265105,391.766824,596778.403014,430.729848,656130.777578,2022-07-26 09:33:16.743594
2,COINBASE,1523.49,109.32,158.146933,240935.271682,249.494542,380102.439727,395.911947,603167.892376,428.431019,652710.372658,2022-07-26 09:41:34.159677
3,FTX,1523.30,109.10,158.096986,240829.138028,249.396040,379904.987237,395.737441,602826.844161,428.431019,652628.970764,2022-07-26 09:41:34.159677
